In [0]:
%pip install holidays

In [0]:
# Importando as funções necessárias
from pyspark.sql import functions as F
from pyspark.sql.types import StringType
import holidays
import datetime

In [0]:

dbutils.widgets.text("data_inicio", "2020-01-01")
dbutils.widgets.text("data_fim", "2030-12-31")

data_inicio = dbutils.widgets.get("data_inicio")
data_fim = dbutils.widgets.get("data_fim")

print(f"Gerando calendário de {data_inicio} até {data_fim}")
print(f"Usando a biblioteca 'holidays' para feriados FEBRABAN.")

In [0]:
df_base_dates = spark.sql(f"SELECT explode(sequence(to_date('{data_inicio}'), to_date('{data_fim}'), interval 1 day)) as data")

In [0]:
#Definição da UDF para Feriados FEBRABAN
br_bancario_holidays = holidays.country_holidays('BR')

In [0]:

def obter_feriado(data_input):
  # O método .get() da biblioteca retorna o nome do feriado ou None se não for um.
  return br_bancario_holidays.get(data_input)

In [0]:
# Registramos a função como uma UDF do Spark, especificando o tipo de retorno.
# Isso permite que a gente a chame como se fosse uma função nativa do Spark.
obter_feriado_udf = F.udf(obter_feriado, StringType())

In [0]:
# Enriquecimento da Tabela com Atributos e Feriados

df_calendario = df_base_dates \
    .withColumn("ano", F.year(F.col("data"))) \
    .withColumn("mes", F.month(F.col("data"))) \
    .withColumn("dia", F.dayofmonth(F.col("data"))) \
    .withColumn("dia_da_semana_num", F.dayofweek(F.col("data"))) \
    .withColumn("dia_do_ano", F.dayofyear(F.col("data"))) \
    .withColumn("semana_do_ano", F.weekofyear(F.col("data"))) \
    .withColumn("trimestre", F.quarter(F.col("data"))) \
    .withColumn("nome_mes_extenso", F.date_format(F.col("data"), "MMMM")) \
    .withColumn("nome_mes_curto", F.date_format(F.col("data"), "MMM")) \
    .withColumn("nome_dia_semana_extenso", F.date_format(F.col("data"), "EEEE")) \
    .withColumn("nome_dia_semana_curto", F.date_format(F.col("data"), "E")) \
    .withColumn("ano_mes_id", F.date_format(F.col("data"), "yyyyMM").cast('int'))


In [0]:
# Adicionando flags de Fim de Semana
df_calendario = df_calendario.withColumn(
    "flag_fim_de_semana",
    F.when(F.col("dia_da_semana_num").isin([1, 7]), "S").otherwise("N")
)

In [0]:
# Aplicamos a UDF na coluna de data para obter a descrição do feriado.
df_com_feriados = df_calendario.withColumn("descricao_feriado", obter_feriado_udf(F.col("data")))


In [0]:
# Criando as Flags Finais e Organizando a Tabela
df_final = df_com_feriados.withColumn(
    "flag_feriado_bancario",
    F.when(F.col("descricao_feriado").isNotNull(), "S").otherwise("N")
)

df_final = df_final.withColumn(
    "flag_dia_util_bancario",
    F.when((F.col("flag_fim_de_semana") == "N") & (F.col("flag_feriado_bancario") == "N"), "S").otherwise("N")
)

In [0]:
# Selecionando e ordenando as colunas para a tabela final
colunas_finais = [
    "data", "ano", "mes", "dia", "trimestre", "ano_mes_id",
    "nome_mes_extenso", "nome_mes_curto", "dia_da_semana_num",
    "nome_dia_semana_extenso", "nome_dia_semana_curto", "dia_do_ano",
    "semana_do_ano", "flag_fim_de_semana", "flag_feriado_bancario",
    "descricao_feriado", "flag_dia_util_bancario"
]

df_resultado = df_final.select(colunas_finais).orderBy("data")

In [0]:
# Salvando a Tabela no Formato Delta
tabela = "rp.bronze.dim_calendario"
df_resultado.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable(tabela)

print("Tabela de calendário criada com sucesso!")